---
author: Delvin Mackenzie
date: 11/19/2019
contact: delvin@uw.edu

---



In [5]:
#Math stuff
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np
import math  

#Visualization stuff
import seaborn as sns
#sns.set(style="darkgrid")
sns.set(style="whitegrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

#ML stuff
from sklearn import preprocessing

In [6]:
#create merge.csv

In [101]:
#check list of files in data 
directory = "./data/dedicated_host/11-16-2019/"
print(os.listdir(directory))

['merged_11-16-2019.csv', 'm5d_pgbench_dedicated_host_11-16-2019.csv', 'm5d_y-cruncher_dedicated_host_11-16-2019_us-east-1 - Sheet1.csv', 'm5d_iperf_dedicated_host_11-17.csv', 'm5d_sysbench_dedicated_host_11-16-2019_us-east-1b - Sheet1.csv']


In [102]:
#import the 4 benchmarks
data_sysbench = 'm5d_sysbench_dedicated_host_11-16-2019_us-east-1b - Sheet1.csv'
data_iperf = 'm5d_iperf_dedicated_host_11-17.csv'
data_y_cruncher = 'm5d_y-cruncher_dedicated_host_11-16-2019_us-east-1 - Sheet1.csv'
data_pgbench ='m5d_pgbench_dedicated_host_11-16-2019.csv'

In [103]:
#TODO: configure header values accordingly...
df_sysbench = pd.read_csv(directory + data_sysbench, header = 1)
df_iperf = pd.read_csv(directory + data_iperf, header = 0)
df_y_cruncher = pd.read_csv(directory + data_y_cruncher, header = 1)
df_pgbench = pd.read_csv(directory + data_pgbench, header = 0)

In [104]:
#create total time for iperf, if needed

#strip trailing text
df_iperf['bandwidthUpload'] = df_iperf['bandwidthUpload'].str.split(" ").str[0].astype('float64')
df_iperf['bandwitdhDownload'] = df_iperf['bandwitdhDownload'].str.split(" ").str[0].astype('float64')
    
#adjust gigbite values 'up' assuming all mbs values are over 100mb
df_iperf['bandwitdhDownload'] = np.where(df_iperf['bandwitdhDownload'] > 100, df_iperf['bandwitdhDownload'], df_iperf['bandwitdhDownload']*1000)
df_iperf['bandwidthUpload'] = np.where(df_iperf['bandwidthUpload'] > 100, df_iperf['bandwidthUpload'], df_iperf['bandwidthUpload']*1000)

#add total bandwidth column
df_iperf['total'] = df_iperf['bandwidthUpload'] + df_iperf['bandwitdhDownload']

#change to GB
#df_iperf['total'] = df_iperf['total'].astype('float64')/1000

In [106]:
#y-cruncher typo in header...adjust it...
#df_y_cruncher['setId'] = df_y_cruncher['setid']

In [107]:
def cleanAndReturnDf(df, experiment_type, col, n_rows = 3, vm_count = 48):
    ''' returns cleaned df with n rows sampled randomly. 
        also drops the first row of each run since they are often outliers.
    '''
    
    df = df.dropna(how='all')
    
    #add set Ids for pgbench,sysbench,y-cruncher
    if (experiment_type != 'iperf'):
        #df = df.dropna()
        #create setId column
        df['setId'] = df['experimentID'].str.strip().str[-2:]
        df['setId'] = df['setId'].astype('int32').abs()
    
    df = df[col]
    
    #sort by setId and instanceID
    df = df.sort_values(by = ['setId','instanceID'], ascending=False)
    
    #iperf doesn't need so skip this part
    if (experiment_type != 'iperf'):
        
        df['vmId'] = pd.factorize(df.instanceID)[0] + 1 #replace instanceID with numerical vmID
        
        df = df.sort_values(by = ['setId','vmId'])
        df = df.drop(columns=['instanceID'])
        
        #finally reverse setId to match eds version! 
        df['setId'] = pd.factorize(df.setId)[0]- vm_count + 1
        #absolute value
        df['setId'] = df['setId'].abs()

        #add + 1 for off by 1 error
        df['setId'] += 1

    df = df.sort_values(by = ['setId','vmId'])
    df = df.reset_index(drop=True)

    #set nth column #change depending on data set...
    if (experiment_type == 'iperf'):
        nth_column = 11
    elif (experiment_type != 'pgbench'):
        nth_column = 10
    
    #drop every nth row
    if (experiment_type != 'pgbench'):
        df = df[df.index % nth_column != 0]
        
        #sample of n_rows to match pgbench row count
        df = df.groupby(['setId', 'vmId']).apply(pd.DataFrame.sample, n=n_rows, random_state=19).reset_index(drop=True)
    
    return df

In [108]:
#create cleaned dataframes for each benchmark
df_iperf_clean = cleanAndReturnDf(df_iperf, 'iperf', ['total','setId','vmId', 'instanceID'])
df_sysbench_clean = cleanAndReturnDf(df_sysbench, 'sysbench', ['setId', 'wallTime', 'instanceID'])
df_pgbench_clean = cleanAndReturnDf(df_pgbench, 'pgbench', ['setId', 'transactions', 'instanceID'])
df_y_cruncher_clean = cleanAndReturnDf(df_y_cruncher, 'y-cruncher', ['setId','wallTime','instanceID'])


In [109]:
#create df for merge.csv
df_merge = df_sysbench_clean

In [111]:
df_merge = df_merge.rename(columns={"wallTime": "sysbench"})

In [112]:
df_merge['iperf'] = df_iperf_clean['total']
df_merge['pgbench'] = df_pgbench_clean['transactions']
df_merge['ycruncher'] = df_y_cruncher_clean['wallTime']

In [113]:
col = ['iperf', 'sysbench', 'ycruncher', 'pgbench', 'setId', 'vmId']
df_merge = df_merge[col]

In [114]:
df_merge.head()

,iperf,sysbench,ycruncher,pgbench,setId,vmId
0,9640.0,7.179230,4.402891,179240,1,1
1,9940.0,7.173129,4.458205,178584,1,1
2,9020.0,7.175981,4.436651,179612,1,1
3,9760.0,7.187608,4.508560,179345,2,1
4,9940.0,7.177871,4.482959,179119,2,1


In [115]:
df_merge.describe()

,iperf,sysbench,ycruncher,pgbench,setId,vmId
count,3528.000000,3528.000000,3528.000000,3528.000000,3528.000000,3528.000000
mean,1871.522392,8.425494,5.794917,143490.297619,32.333333,16.666667
std,1618.387464,0.547367,0.557579,15952.650512,11.427711,11.427711
min,478.000000,7.170404,4.372757,114134.000000,1.000000,1.000000
25%,1101.000000,7.782930,5.457215,128668.750000,24.000000,7.000000
50%,1392.000000,8.771970,6.025283,138478.000000,34.000000,15.000000
75%,1859.000000,8.790618,6.290605,158502.500000,42.000000,25.000000
max,9940.000000,10.484629,7.109713,184283.000000,48.000000,48.000000


In [116]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528 entries, 0 to 3527
Data columns (total 6 columns):
iperf        3528 non-null float64
sysbench     3528 non-null float64
ycruncher    3528 non-null float64
pgbench      3528 non-null int64
setId        3528 non-null int64
vmId         3528 non-null int64
dtypes: float64(3), int64(3)
memory usage: 165.5 KB


In [118]:
#print to csv
df_merge.to_csv(path_or_buf= directory + 'merged_11-16-2019.csv')

In [52]:
# col = ['setId', 'wallTime', 'instanceID']

In [58]:
#filter by relevant columns
# df = df_sysbench[col]

In [61]:
# df.dropna(inplace=True)

In [59]:
#sort by setId and instanceID
# df.sort_values(by = ['setId','instanceID'], ascending=False, inplace=True)

In [60]:
#replace instanceID with numerical vmID
# df['vmId'] = pd.factorize(df.instanceID)[0] + 1

In [62]:
#sort by vmId then setId 
# df.sort_values(by = ['setId','vmId'], inplace=True)

In [64]:
#drop instanceID
# df.drop(columns=['instanceID'], inplace=True)

In [68]:
# #finally reverse setId to match eds version! 
# #change 48 if necessary to match vm count
# vm_count = 48
# df['setId'] = pd.factorize(df.setId)[0]- vm_count + 1
# #absolute value
# df['setId'] = df['setId'].abs()

In [72]:
#add + 1 for off by 1 error
# df['setId'] += 1

In [73]:
# df.sort_values(by = ['setId','vmId'], inplace=True)
# df.reset_index(drop=True, inplace=True)

In [75]:
# experiment_type = 'sysbench'

In [76]:
# #drop every nth row...
# #set nth column #change depending on data set...
# if (experiment_type == 'iperf'):
#     nth_column = 11
# #     if (not dedicated_host_mode):
# #         df_data = df_data[:551] #drop every run except first run
# elif (experiment_type != 'pgbench'):
#     nth_column = 10

In [77]:
# #drop every nth column
# #drop first row of each vm run since they are outliers (except pgbench)
# if (experiment_type != 'pgbench'):
#     df = df[df.index % nth_column != 0]

In [79]:
# #take a sample of 3 for the 3 bench marks...
# #drop all but 3 rows? since pgbench only has 3 rows?
# n_rows = 3
# df = df.groupby(['setId', 'vmId']).apply(pd.DataFrame.sample, n=n_rows, random_state=19).reset_index(drop=True)

In [158]:
# df.head()

In [ ]:
#drop all but 3 rows? since pgbench only has 3 rows?
#df.groupby('Group_Id').apply(pd.DataFrame.sample, n=1).reset_index(drop=True)

In [ ]:
#export to csv